# Import Library

In [15]:
import os
import numpy as np
import math
import cv2

# Data Path

In [16]:
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
train_path = "./images/train"
test_path = "./images/test"
model_path = "./face_recognizer_model.xml"

# Menu 1

In [17]:
def menu1():

    person_names = os.listdir(train_path)
    class_list = []
    face_list = []

    for idx, name in enumerate(person_names):
        img_path = os.path.join(train_path, name)
        for img_name in os.listdir(img_path):
            full_img_path = os.path.join(img_path, img_name)
            img_gray = cv2.imread(full_img_path, cv2.IMREAD_GRAYSCALE)
            detected_faces = face_cascade.detectMultiScale(img_gray, scaleFactor=1.2, minNeighbors=5)

            if len (detected_faces) < 1:
                continue

            for rect in detected_faces:
                x, y, w, h = rect
                face_img = img_gray[y:y+w, x:x+h]
                face_list.append(face_img)
                class_list.append(idx)

    face_recognizer.train(face_list, np.array(class_list))
    face_recognizer.save(model_path)

    correct_pred = 0
    total_pred = 0

    classes = os.listdir(train_path)

    for label, name in enumerate(classes):
        img_path = os.path.join(test_path, name)
        for img_name in os.listdir(img_path):
            full_img_path = os.path.join(img_path, img_name)
            img_gray = cv2.imread(full_img_path, cv2.IMREAD_GRAYSCALE)
            detected_faces = face_cascade.detectMultiScale(img_gray, scaleFactor=1.2, minNeighbors=5)

            if len (detected_faces) < 1:
                continue

            for rect in detected_faces:
                x, y, w, h = rect
                face_img = img_gray[y:y+w, x:x+h]
                prediction, _ = face_recognizer.predict(face_img)
                if prediction == label:
                    correct_pred += 1
                total_pred += 1
    
    print (f"Accuracy : {correct_pred/total_pred*100}%")

# Menu 2

In [ ]:
def menu2(img_path):

    if not os.path.exists(model_path):
        print ("Train model first")
        return
    
    face_recognizer.read(model_path)

    person_names = os.listdir(train_path)

    test_img = img_path

    if test_img is None:
        print ("The image path is not correct")
        return

    img_bgr = cv2.imread(test_img)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_RGB2GRAY)
    detected_faces = face_cascade.detectMultiScale(img_gray, scaleFactor=1.2, minNeighbors=5)

    for rect in detected_faces:
        x, y, w, h = rect
        face_img = img_gray[y:y+w, x:x+h]
        label, confidence = face_recognizer.predict(face_img)

        confidence = math.floor(confidence*100)/100
        name = person_names[label]
        text = name + " " + str(confidence)

        cv2.rectangle(img_bgr, (x, y), (x+w, y+h), (0, 255, 255), 1)
        cv2.putText(img_bgr, text, (x, y-10), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0))
    cv2.imshow("Result", img_bgr)
    cv2.waitKey(0)

# Main Menu

In [19]:
def print_menu():
    print ("1. Train Model")
    print ("2. Predict Image")
    print ("3. Exit")

In [20]:
def main_menu():
    choice = -1
    while True:
        print_menu()
        choice = int(input("Input  Menu: "))
        if choice == 1:
            menu1()
        elif choice == 2:
            path = input("Put absolute path: ")
            menu2(path)
        elif choice == 3:
            break
        else:
            print ("Invalid Menu!")

    print ("Thanks")

In [21]:
main_menu()

1. Train Model
2. Predict Image
3. Exit
1. Train Model
2. Predict Image
3. Exit
Accuracy : 50.0%
1. Train Model
2. Predict Image
3. Exit
Thanks
